In [582]:
import sys
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import random
from typing import List, Any, Dict, Tuple
import copy
from datetime import datetime
from collections import Counter, defaultdict
from scipy import stats
from tabulate import tabulate

sys.path.append("../")

from src.scripts import run_fake_data_test
from src.helpers import visualisation # import barplot_distribution, plot_confusion_matrix, tabulate_annotation_pair_summary, analyze_pair_annotations
from src.helpers.io import read_json


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [583]:
# !pip3 install statsmodels

In [584]:
prompt_fields_prev = [
    "multi_turn_relationship",
    "media_format",
    "topic",
    "function_purpose",
    "anthropomorphization",
    "restricted_flags",
]
response_fields_prev = [
    "answer_form",
    "self_disclosure",
    "topic_response",
    "media_format_response",
    "restricted_flags_response",
]

prompt_fields_new = [
    "prompt_multi_turn_relationship",
    "prompt_media_format",
    "prompt_interaction_features",
    "prompt_function_purpose",
    "turn_topic",
    "turn_sensitive_use_flags",
]
response_fields_new = [
    "response_answer_form",
    "response_media_format",
    "response_interaction_features",
]

In [598]:
dset = run_fake_data_test.run_automatic_analysis_v0("../data/")

Loading conversations from ../data/sample120.json
Loaded 120 conversations.
Updated file: shayne60_annotations.json
Added conversation IDs to 1 files
Split records into two folders:
  - ../data/labelstudio_outputs_split1_v3/: Contains 60 records with unique conversation IDs
  - ../data/labelstudio_outputs_split2_v3/: Contains 0 records with duplicate conversation IDs

gpt4o-json

prompt-multi_turn_relationship: 3 / 597 failed due to invalid annotations.
prompt-interaction_features: 1 / 597 failed due to invalid annotations.
turn-sensitive_use_flags: 0 / 597 failed due to invalid annotations.
turn-topic: 11 / 597 failed due to invalid annotations.
response-interaction_features: 0 / 597 failed due to invalid annotations.
prompt-function_purpose: 12 / 597 failed due to invalid annotations.
prompt-media_format: 12 / 597 failed due to invalid annotations.
response-media_format: 4 / 597 failed due to invalid annotations.
response-answer_form: 2 / 597 failed due to invalid annotations.

gpt4o

In [599]:
# task_annotations = {}
# for feature in response_fields_new:
#     task_annotations[feature] = visualisation.run_interrater_comparison(dset, feature, "gpt4o_json", "gpt4o_free", "../data/annotation_analysis_v0")
#     # break

In [600]:
task_annotations = {}
for feature in prompt_fields_new + response_fields_new:
    task_annotations[feature] = visualisation.run_interrater_comparison(dset, feature, "gpt4o_json", "split1", "../data/annotation_analysis_v1")
    # break

../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')


Found 594 items with `gpt4o_json-prompt_multi_turn_relationship`, and 146 with `split1-prompt_multi_turn_relationship`.
Generated 146 label-level pairs (at level=message) out of 1195 total items.

-----------------prompt_multi_turn_relationship-----------------
{'label_agreement_rate': 0.733, 'cohens_kappa': 0.641, 'krippendorffs_alpha': 0.646, 'jaccard_similarity': 0.74, 'f1_score': 0.742}
+---------------------------------------+---------------------------------+---------+------------------+------------+
| val1                                  | val2                            |   count |   percent_of_all | matching   |
+=======================================+=================================+=========+==================+============+
| ['First Request']                     | ['First Request']               |      60 |            41.1  | T          |
+---------------------------------------+---------------------------------+---------+------------------+------------+
| ['Completely N

../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')


Found 585 items with `gpt4o_json-prompt_media_format`, and 146 with `split1-prompt_media_format`.
Generated 141 label-level pairs (at level=message) out of 1195 total items.

-----------------prompt_media_format-----------------
{'label_agreement_rate': 0.411, 'cohens_kappa': 0.165, 'krippendorffs_alpha': 0.2, 'jaccard_similarity': 0.708, 'f1_score': 0.802}
+-----------------------------------------------------------------------+----------------------------------------------------------------+---------+------------------+------------+
| val1                                                                  | val2                                                           |   count |   percent_of_all | matching   |
+=======================================================================+================================================================+=========+==================+============+
| ['Natural Language']                                                  | ['Natural Language']   

../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')
../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')


...

Found 595 items with `gpt4o_json-response_answer_form`, and 146 with `split1-response_answer_form`.
Generated 146 label-level pairs (at level=message) out of 1195 total items.

-----------------response_answer_form-----------------
{'label_agreement_rate': 0.801, 'cohens_kappa': 0.385, 'krippendorffs_alpha': 0.391, 'jaccard_similarity': 0.815, 'f1_score': 0.82}
+-----------------------------------------------+-----------------------------------+---------+------------------+------------+
| val1                                          | val2                              |   count |   percent_of_all | matching   |
+===============================================+===================================+=========+==================+============+
| ['Direct Answer Open Generation']             | ['Direct Answer Open Generation'] |     108 |            73.97 | T          |
+-----------------------------------------------+-----------------------------------+---------+------------------+----------

../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')


Found 593 items with `gpt4o_json-response_media_format`, and 146 with `split1-response_media_format`.
Generated 145 label-level pairs (at level=message) out of 1195 total items.

-----------------response_media_format-----------------
{'label_agreement_rate': 0.738, 'cohens_kappa': 0.613, 'krippendorffs_alpha': 0.646, 'jaccard_similarity': 0.881, 'f1_score': 0.921}
+---------------------------------------------------+---------------------------------------------------------------+---------+------------------+------------+
| val1                                              | val2                                                          |   count |   percent_of_all | matching   |
+===================================================+===============================================================+=========+==================+============+
| ['Natural Language']                              | ['Natural Language']                                          |      57 |            39.31 | T    

../src/helpers/visualisation.py:125: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  fig.savefig(output_path, bbox_inches='tight')


Found 597 items with `gpt4o_json-response_interaction_features`, and 146 with `split1-response_interaction_features`.
Generated 146 label-level pairs (at level=message) out of 1195 total items.

-----------------response_interaction_features-----------------
{'label_agreement_rate': 0.863, 'cohens_kappa': 0.585, 'krippendorffs_alpha': 0.622, 'jaccard_similarity': 0.885, 'f1_score': 0.891}
+-------------------------------------------------+------------------------------------------------------------------------+---------+------------------+------------+
| val1                                            | val2                                                                   |   count |   percent_of_all | matching   |
+=================================================+========================================================================+=========+==================+============+
| ['None']                                        | ['None']                                               

In [471]:
convos = dset.find_conversations_by_metadata(
    annotation_key=("gpt4_json", "response_interaction_features"),
    level = "messages",
    search_list = None
)

In [473]:
# convos

## Formatted Inspection of Annotations for given Tasks and ExID

In [154]:
ex_ids = []
orig_sample = read_json("../data/sample120.json")
for datum in orig_sample["data"]:
    for turn in datum["conversation"]:
        ex_ids.append(datum["conversation_id"] + "-" + str(turn["turn"]))

In [353]:
ANNOTATION_TURN = 304
visualisation.display_info_for_turn(dset, ex_ids[ANNOTATION_TURN], prompt_fields_new + response_fields_new)

IDX: wildchat_971feda71014f7166280fc1b8553cb32 | Turn: 4 | Role: user
-------------------------------------------

TASK: prompt_multi_turn_relationship
gpt4o_json:   ['Completely new request']
gpt4o_free:   ['Completely new request']
gpto3mini_json:   ['Completely new request']
gpto3mini_free:   ['Completely new request']

TASK: prompt_interaction_features
gpt4o_json:   ['None']
gpt4o_free:   ['None']
gpto3mini_json:   ['None']
gpto3mini_free:   ['None']

TASK: turn_sensitive_use_flags
gpt4o_json:   ['None']
gpt4o_free:   ['None']
gpto3mini_json:   ['None']
gpto3mini_free:   ['None']

TASK: turn_topic
gpt4o_json:   ['Food & Dining', 'Entertainment, Hobbies & Leisure']
gpt4o_free:   ['Food & Dining', 'Entertainment, Hobbies & Leisure']
gpto3mini_json:   ['Food & Dining']
gpto3mini_free:   ['Food & Dining']

TASK: response_interaction_features
gpt4o_json:   ['None']
gpt4o_free:   ['None']
gpto3mini_json:   ['None']
gpto3mini_free:   ['None']

TASK: prompt_function_purpose
gpt4o_json:   [

## Inspect a single message

In [493]:
convos[-2]

'sharegpt_3fTHHS8-12'

In [490]:
# EG_ID = "wildchat_40fe9070a5268327e0278d00a7bd1396-0"
EG_ID = convos[-2]

In [491]:
message = dset.id_lookup(EG_ID, level="message")[EG_ID].to_dict()

In [492]:
message

{'turn': 12,
 'role': 'human',
 'content': "Can I quote you in an article I'm going to blog? How would I cite you?",
 'timestamp': None,
 'metadata': {'gpt4o_json-prompt_multi_turn_relationship': {'value': ['Completely New Request'],
   'confidence': [0.85],
   'target_id': 'sharegpt_3fTHHS8-12',
   'annotator': None},
  'gpt4o_json-prompt_interaction_features': {'value': ['None'],
   'confidence': [0.9],
   'target_id': 'sharegpt_3fTHHS8-12',
   'annotator': None},
  'gpt4o_json-turn_sensitive_use_flags': {'value': ['Output Misrepresentation Expressly Disclaiming AI Use or Source Material'],
   'confidence': [0.8],
   'target_id': 'sharegpt_3fTHHS8-12',
   'annotator': None},
  'gpt4o_json-turn_topic': {'value': ['Literature & Writing',
    'Technology, Software & Computing'],
   'confidence': [0.7, 0.6],
   'target_id': 'sharegpt_3fTHHS8-12',
   'annotator': None},
  'gpt4o_json-prompt_function_purpose': {'value': ['Information Retrieval: General Info From Prompt Content',
    'Edito

## See set of all annotation labels from each source

In [328]:
relevant_task_keys = prompt_fields_new + response_fields_new
for task_key in relevant_task_keys:
    
    human_annotated_vals = dset.extract_all_metadata_values_for_annotation_source_name(
        [("split1", task_key), ("split2", task_key)],
        level="message",
    )
    automatic_annotated_vals = dset.extract_all_metadata_values_for_annotation_source_name(
        [("gpt4o_free", task_key), ("gpt4o_json", task_key), ("gpto3mini_json", task_key), ("gpto3mini_free", task_key)],
        level="message",
    )
    print("------------------------------------------------")
    print(task_key)
    print()
    print(human_annotated_vals)
    print()
    print(automatic_annotated_vals)
    print("------------------------------------------------")

1195
1195
------------------------------------------------
prompt_multi_turn_relationship

{'New variation of prior task', 'Re-attempt / revision on prior request', 'Extend, deepen, or build on prior task', 'Completely new request', 'First request'}

{'New variation of prior task', 'Re-attempt/revision on prior task', 'Extend, deepen, or build on prior task', 'Completely new request', 'First request'}
------------------------------------------------
1195
1195
------------------------------------------------
prompt_media_format

{'Code', 'Natural language', 'Other', 'Images', 'Math / symbols', 'Formatted enumeration / itemization', 'Likely retrieved / pasted content', 'Charts / graphs', 'URLs', 'HTML'}

{'Code', 'Charts/Graphs', 'Natural language', 'Images', 'Other', 'Formatted enumeration/itemization (bullets/lists)', 'Formatted enumeration/itemization', 'Math/symbols', 'URLs', 'URL', 'Likely retrieved/pasted content', 'HTML'}
------------------------------------------------
1195
1195
